# P06 - Avis Restau - Détecter les "bad buzz" laissés dans les commentaires concernant un restaurant

# 0. Import des librairies

In [8]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 200)

In [9]:
import json
import tarfile
import pickle

In [10]:
# NLP
import re
import nltk
#nltk.download()
import spacy
import gensim
import pprint

# 1. Analyse et traitements des fichiers

import os
path = ~/file
path

In [11]:
#my_tar = tarfile.open('yelp_dataset.tar'
#my_tar.extractall()
#my_tar.close()

Après cette étpade de décompression, nous nous retrouvons avec 5 fichiers:

In [12]:
file_business = 'yelp_academic_dataset_business.json'
file_checking = 'yelp_academic_dataset_checkin.json'
file_review = 'yelp_academic_dataset_review.json'
file_tip = 'yelp_academic_dataset_tip.json'
file_user = 'yelp_academic_dataset_user.json'

Je commence par regarder la nature de chaque fichier (les différentes variables) pour voir quelles seront les informations à conserver dans le cadre de mon travail, à savoir de détecter les "bad buzz" concernant chaque restaurant.  
Cela passera par deux aspects:  
    - identifier les établissements qui sont bel et bien des restaurants.  
    - trier les bons et mauvais avis en fonction de chaque note. Cette dernière allant de 1 à 5, je me limiterai aux avis ayant une note égale à 1 ou 2  
Une fois cela fait, alors je pourrai me pencher plus en détail sur les commentiares en question

Déjà, j'en regarde la longueur. En effet, sachant que l'ensemble des 5 fichiers prend plus de 10 Go en mémoire, il est plus que probable qu'il faudra ne garder qu'une portion de chaque fichier

In [6]:
def count_line_file(file):
    line_count = 0
    with open(file, 'r') as f:
        while f.readline():
            line_count+=1
    return line_count

In [7]:
print('le fichier business possède',count_line_file(file_business), 'lignes')
print('le fichier checking possède',count_line_file(file_checking), 'lignes')
print('le fichier review possède',count_line_file(file_review), 'lignes')
print('le fichier tip possède',count_line_file(file_tip), 'lignes')
print('le fichier user possède',count_line_file(file_user), 'lignes')

le fichier business possède 160585 lignes
le fichier checking possède 138876 lignes


OSError: [Errno 89] Operation canceled

On voit qu'il y a 3 fichiers très volumineux (review, tip et user) qui, chacun, possède plus d'1 million de lignes

Maintenant, pour avoir une idée plus précise des informations (variables) de chaque fichier j'en regarde la première ligne dans un dataframe:

In [ ]:
### Je définis une fonction qui me permettra d'obtenir une liste contenant les n premières lignes d'un fichier

def file_to_df_first_n_lines(file, n):
    liste = []
    df = pd.DataFrame()
    with open(file, 'r') as f:
        for i in range(n):
                line = f.readline()
                dico = json.loads(line)
                df = df.append(dico, ignore_index=True )
    return df

J'affiche la 1ère ligne de chaque dataframe obtenu

In [ ]:
file_to_df_first_n_lines(file_business, 1)

Ici, éventuellement le business_id, le nom, le nombre d'étoiles (stars) et la catégorie seront à garder

In [ ]:
file_to_df_first_n_lines(file_checking, 1)

... Pour ce fichier, aucune information utile!

In [ ]:
file_to_df_first_n_lines(file_review, 1)

Ici, pour pouvoir ne garder que les avis des restaurants, il faudra garder:  
    - le business_id      
    - le user_id    
    - "stars"   
    - et évidemment, "text"

In [ ]:
file_to_df_first_n_lines(file_tip, 1)

"text" (en tant que "tip") pourrait aussi être important pour nous (éventuellement) donc faire le lien avec le resrte je garderai "user_id", "business_id" et "text"

In [ ]:
file_to_df_first_n_lines(file_user, 1)

Là il ne s'agit que de la fiche d'information d'un utilisateur, qui ne sera pas nécessaire pour la suite

A présent, je vais construire les dataframes selon les informations qui nous serviront part la suite:  

In [ ]:
### Je définis la fonction qui, à partir d'un fichier:
### ne gardera que n lignes
### et se conterera, pour chaque ligne JSON du fichier, de n'en garder que les clés importantes (columns)
### pour finalement transfomer ce résultat en un dataframe

def file_to_df(file, n, columns):
    df = pd.DataFrame()
    with open(file, 'r') as f1:
        for i in range(n):
            line = f1.readline()
            dico = json.loads(line)
            dico = {c:dico[c] for c in columns}
            df = df.append(dico, ignore_index = True)
    return df

In [ ]:
#%%time
#df_business = file_to_df(file_business, count_line_file(file_business), ['business_id', 'stars', 'categories'])

**14 minutes** pour traiter **160 000 lignes**... Pour les 2 fichiers qu'il me reste (file_review et file_tip), je me contenterai donc de ne garder que **150 000** lignes chacun... et pour éviter de relancer la cellule ultérieurement, j'en fais une sauvegarde avec le module "pickle"

In [ ]:
# je sauvegarde le dataframe précédent

file = open('df_business_saved', 'wb')
pickle.dump(df_business, file)
file.close()


In [ ]:
# ... que je recharge:
file = open('df_business_saved', 'rb')
df_business = pickle.load(file)
file.close()

In [ ]:
#%%time
#df_review = file_to_df(file_review, 150000, ['user_id', 'business_id', 'stars', 'text'])

In [ ]:
# je sauvegarde le dataframe précédent

file = open('df_review_saved', 'wb')
pickle.dump(df_review, file)
file.close()

In [ ]:
# ... que je recharge:
file = open('df_review_saved', 'rb')
df_review = pickle.load(file)
file.close()

In [ ]:
%%time
#df_tip = file_to_df(file_tip, 150000, ['user_id','business_id', 'text'])

In [ ]:
df_business.head()

In [ ]:
df_review

In [ ]:
#df_tip.head()

In [ ]:
df_business  = df_business[df_business['categories'].notnull()]

In [ ]:
df_restaurant = df_business[df_business['categories'].str.contains('Restaurants')]

In [ ]:
df_business['categories'].str.contains('Restaurants').isnull().sum()

In [ ]:
df_restaurant.head()

In [ ]:
df_restaurants_reviews = df_review[df_review['business_id'].isin(df_restaurant['business_id'])]

In [ ]:
df_restaurants_reviews.shape, df_review.shape

In [ ]:
df_restaurants_reviews = df_restaurants_reviews[df_restaurants_reviews['stars']<3]

In [ ]:
df_restaurants_reviews

In [ ]:
nb_restaurants = df_restaurants_reviews['business_id'].nunique()
print('il y a ', nb_restaurants, 'restaurants différents parmi les 20440 mauvais commentaires (note<3) laissés par les clients')

In [ ]:
# Je sauvegarde ces résultats (le dataframe obtenu), dans un fichier: peut-être nous sera-t-il utile pour la suite...
df_restaurants_reviews.to_csv('restaurants_bad_reviews.csv', index=False)

# 2. Traitement des données textuelles - NLP

Maintenant que nous avons gardé:  
    - les restaurants parmi les différents établissements.  
    - et au sein de ceux-ci, seuls les restaurants ayant reçu des mauvais avis (c'est à dire des notes <3)  
Alors on peut s'attaquer à la partie "traitement du langage", appelé en anglais NLP (Natural langage processing)

## 2.1. Normalisation des commentaires

Une des premières étapes pour ce faire est la NORMALISATION des données: il s'agit de convertir chaque mot dans une forme **canonique** en leur appliquant certaines opérations:
- la **tokenisation** (à savoir "éclater" le texte en différents mots)
- enlever ce qu'on appelle les **"stop words"**, mots qui sont bien sûr utiles dans la compréhension du discours humain, mais pas pour le NLP 
- enfin, le **POS-tagging (POS pour part of speech)** et la **lemmatisation**, qui consistent  à ramener un mot à sa racine grammaticale (le vocable)


### 2.1.1 Nettoyage des données: tokenisation, enlèvement des "stop words"

In [ ]:
reviews = pd.read_csv('restaurants_bad_reviews.csv')

In [ ]:
nb_reviews = reviews.shape[0]

In [ ]:
list_of_reviews = reviews['text'].values.tolist()
# list_of_reviews est une liste contenant nb_reviews (donc 20440) éléments qui sont eux des chaînes de caractères (str)

la tokenisation et la suppression des "stop words"se feront grâce aux méthodes de la librairie [NLTK]('https://www.nltk.org/')  

In [ ]:
# Exemple:
comment = reviews.iloc[2]['text']
print(comment)

In [ ]:
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

def normalisation(text):
    #transforme le texte en tokens
    tokens = word_tokenize(text)
    #convertit les majuscules en minusucles
    tokens = [word.lower() for word in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words
    

In [ ]:
print(normalisation(comment))

la fonction "normalisation" a donc transformé le commentaire en une liste ne contenant plus que les "tokens", sans les "stop words"

A présent, nous pouvons appliquer cette fonction sur l'ensemble des commentaires de notre "list_of_reviews"

In [ ]:
list_of_reviews_normalized = [normalisation(elt) for elt in list_of_reviews]

In [ ]:
print(list_of_reviews_normalized[2], end= ' ')

### 2.1.2 POS tagging et lemmatisation

le **POS (part of speech) tagging** consiste à identifier les mots selon leur fonction dans la phrase (nom, adjectif, verbe...)
Quant à la **lemmatisation**, elle ramène les mots à leur forme canonique

Pour Les deux, j'utiliserai les méthodes embarquées par la librairie "[**SpaCy**](https://spacy./)"

**Remarque:** ainsi que mentionné ici [Machine learning mastery](https://machinelearningmastery.com/gentle-introduction-bag-words-model/ ),  
"A more sophisticated approach is to create a vocabulary of grouped words. This both changes the scope of the vocabulary and allows the bag-of-words to capture a little bit more meaning from the document."  

Donc, pour pouvoir avoir une meilleure analyse des mauvais commentaires, il apparait judicieux de prévoir, pour notre construction de **"bag of words"** future, de considérer ce qu'on appelle un bigramme (2-gram en anglais) qui nous permettra d'isoler une expression telle que "mauvais service"

In [ ]:
import spacy
import en_core_web_sm #import du modèle "en_core_web_sm" du vocabulaire anglais, de sa syntaxe et de sesn entités
nlp = en_core_web_sm.load() #chargement de ce modèle

In [ ]:
def list_to_string (liste):
    return (' '.join(liste))

In [ ]:
# je crée la liste des TAGs pour notre futur bigram

TAG_list = ['NOUN'] #on ne garde que les noms

In [ ]:
'''je définis une fonction qui prend en entrée un texte et retourne les NOMS COMMUNS contenus dans ce texte
 la fonction:
     -> prend en entrée une chaïne de caractères
     -> renvoie une chaîne de caractères'''

def POS_tagging (text):
    tag = nlp(text)
    l = [word.text for word in tag if word.pos_ in TAG_list]
    return ' '.join(l)

In [ ]:
POS_tagging(comment)

In [ ]:
'''je définis une fonction qui prend en entrée un texte et retourne les mots du texte sous forme canonique (leur "lemme")
 la fonction:
     -> prend en entrée une chaïne de caractères
     -> renvoie une chaîne de caractères'''

def lemmatisation (text):
    lem = nlp(text)
    l = [word.lemma_ for word in lem]
    return ' '.join(l)

A présent on peu enchaîner les deux fonctions: testons les sur le commentaire défini précédemment

In [ ]:
print ('voici le commentaire en question:\n\n', comment)
text = POS_tagging(comment)
text = lemmatisation(text)
print('\n\nvoici le commentaire dont il ne reste que les noms communs ayant été par la suite lemmatisés\n', text)

il ne nous reste plus qu'à transformer tous nos commentaires de notre "list_of_reviews_normalized".  
**Rq** : Cette "list_of_reviews_normalized" étant justement une liste quand il nous faut une chaîne de caractères pour le POS_Tagging, alors il me faudra effectuer cette première étape
    
    

In [ ]:
%%time
cleaned_list = []
for i in range(nb_reviews):
    liste = list_of_reviews_normalized[i]
    text = ' '.join(liste)
    text = POS_tagging(text)
    text = lemmatisation(text)
    cleaned_list.append(text)
    

In [ ]:
reviews['cleaned_text'] = pd.DataFrame(cleaned_list)

In [ ]:
reviews.head()

**Remarque importante!!!** En ne gardant que les noms, on voit que l'on perd beaucoup d'informations. Je vais donc refaire la procédure en incluant cette fois les adjectifs.

In [ ]:
%%time
TAG_list = ['NOUN', 'ADJ'] # cette fois j'inclue les adjectifs

cleaned_list = []
for i in range(nb_reviews):
    liste = list_of_reviews_normalized[i]
    text = ' '.join(liste)
    text = POS_tagging(text)
    text = lemmatisation(text)
    cleaned_list.append(text)


In [ ]:
reviews['cleaned_text'] = pd.DataFrame(cleaned_list)
reviews.head()

### 2.2. Création du "bag of words" avec Gensim

Tout ce qui suit est inspiré de cette [page](https://getdoc.wiki/Gensim-creating-a-bag-of-words-corpus)

In [ ]:
cleaned_list_tokenized = [word_tokenize(elt) for elt in reviews['cleaned_text']]
print(cleaned_list_tokenized[2], end = ' ')

In [ ]:
%%time
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

#Je crée un objet "dictionnary"
id2word = corpora.Dictionary(cleaned_list_tokenized)

# Je crée à présent un corpus de "bag of words" ainsi:
corpus = [id2word.doc2bow(doc, allow_update=True) for doc in cleaned_list_tokenized]
#print(BOW_corpus)

In [ ]:
print(corpus[0])
print(len(corpus[0]))
print(len(cleaned_list_tokenized[0]))

Ici, on voit que notre 1ère ligne:  
"many marriott huge disappointment front desk atrium nice starbuck site nice room old flat screen hotel priceline rate good deal price true renaissance"  
a été transformée en une liste de couples d'éléments où:  
le 1er terme est la position du mot dans la phrase. 
le 2nd est son nombre d'occurences. 

ex: (3,1) représente le mot "dissapointment" et n'apparaît qu'une fois.  

Remarque: nous avons (11,2) ce qui "déplace d'un cran" les autres termes vers la gauche et c'est pourquoi dans corpus[0] nous avons 22 éléments quand nous en avions 23 dans le texte d'origine


Le problème de notre "BoW_corpus" est qu'il n'est pas lisible en l'état mais avec une ligne de code en plus, on peut remédier à cela:

In [ ]:
%%time
id_words = [[(dictionary[id], count) for id, count in line] for line in BoW_corpus]
print(id_words[0])

Maintenant, nous avons notre **corpus** que nous pouvons enregistrer pour la suite

In [ ]:
dictionary.save('dictionary')
corpora.MmCorpus.serialize('corpus', BoW_corpus)

In [ ]:
id2word = dictionary.id2token
temp = dictionary[0]

In [ ]:
corpus = corpora.MmCorpus('corpus')

In [ ]:
len(id2word)

In [ ]:
%%time
from gensim.models import LdaModel

# Training parameters
num_topics = 10
chunksize = len(id2word)
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


Pour la suite, voir [ici](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=model, texts=cleaned_list_tokenized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics = False)
vis

In [ ]:
%%time
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(3, 22, 3):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=cleaned_list_tokenized, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=cleaned_list_tokenized, start=3, limit=22, step=3)
# Show graph
import matplotlib.pyplot as plt
limit=22; start=3; step=3;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

On voit que le meilleur nombre de topics est de 15

In [ ]:
%%time
from gensim.models import LdaModel

# Training parameters
num_topics = 15
chunksize = len(id2word)
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics = False)
vis